### Ingestion de la carpeta "production_country"


In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Leer los archivos JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [0]:
productions_countries_schema = StructType(fields=[
     StructField("movieId", IntegerType(), True),
     StructField("countryId", IntegerType(), True),
])

In [0]:

productions_countries_df = spark.read \
                   .schema(productions_countries_schema) \
                   .option("multiLine", True) \
                   .json(f"{bronze_folder_path}/{v_file_date}/production_country") 

### Paso 2 - Renombrar las columnas y añadir columnas nuevas
1. "movieId" renombrar a "movie_id"
2. "countryId" renombrar a "country_id"
3. Agregar las columnas de "ingestion_date" y "envoronment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
productions_countries_final_df = add_ingestion_date(productions_countries_df) \
                                 .withColumn("environment", lit(v_environment)) \
                                 .withColumn("file_date",lit(v_file_date)) \
                                 .withColumnsRenamed(
                                    {"movieId": "movie_id", 
                                    "countryId": "country_id"
                                    })
                         

### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(productions_countries_final_df,"movie_silver","productions_countries", "file_date")

In [0]:
#productions_countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/productions_countries")

# Crear una tabla en base al data frame productions_countries_final_df
# Creara la tabla productions_countries en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver

#productions_countries_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.productions_countries")

# Append, agrega los registros a la tabla existente, carga incremental de datos

#productions_countries_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_countries")

#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(productions_countries_final_df,"movie_silver","productions_countries",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
-- Consultar la DB movie_silver, tabla productions_countries
SELECT file_date, COUNT(1)
FROM movie_silver.productions_countries
GROUP BY file_date;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/productions_countries"))
display(spark.read.format("delta").load(f"{silver_folder_path}/productions_countries"))

In [0]:
dbutils.notebook.exit("Success")